## Historico a Datalake en Azure

#### instalacion de bibliotecas necesarias para subir datasets desde webscrapping

In [1]:
pip install azure-storage-blob

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


#### Spark para poder descargar gran volumen de datos a traves de librería PySpark

In [5]:
from pyspark.sql import SparkSession
import urllib
import os
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Inicializar la sesión de Spark
spark = SparkSession.builder \
    .appName("Load NYC Taxi Data") \
    .getOrCreate()

# Configuración de las credenciales de Azure Blob Storage
account_name = 'dbstorageceyfyf6fyuxbw'
account_key = 'rH40G/3urTHoJ2R58YOKlQQ0TAQyIavk19Si1ZgLXNiAyiiLx6N31L/Iu47wiF4vbavSgROh+RXF+AStoik10A=='
container_name = 'taxisbucket'

# Crear un cliente Blob para interactuar con el contenedor de Azure
blob_service_client = BlobServiceClient.from_connection_string(f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net")
container_client = blob_service_client.get_container_client(container_name)

# Iterar sobre los meses desde enero de 2021
año = 2021
mes = 1
while True:
    # Construir el nombre del archivo Parquet
    nombre_archivo = f"yellow_tripdata_{año}-{mes:02}.parquet"

    # Construir la ruta del archivo Parquet
    ruta_archivo = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{nombre_archivo}"
    print(f"Intentando cargar el archivo: {ruta_archivo}")  # Impresión para depurar

    try:
        # Nombre del blob en Azure Blob Storage (mismo que el nombre del archivo local)
        blob_name = "/taxis_yellow/" + nombre_archivo

        # Obtener el blob client para el archivo en Azure Blob Storage
        blob_client = container_client.get_blob_client(blob_name)

        # Subir el archivo directamente desde la URL al blob storage
        with urllib.request.urlopen(ruta_archivo) as response:
            blob_client.upload_blob(response.read(), overwrite=True)

        print(f"Archivo '{nombre_archivo}' subido exitosamente a Azure Blob Storage.")
    except Exception as e:
        print(f"No se pudo cargar el archivo {nombre_archivo}: {str(e)}")
        break  # Detener el bucle si no se puede cargar el archivo

    # Pasar al siguiente mes
    mes += 1
    if mes > 12:
        mes = 1
        año += 1

# Detener la sesión de Spark al finalizar
spark.stop()


Intentando cargar el archivo: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet
Archivo 'yellow_tripdata_2021-01.parquet' subido exitosamente a Azure Blob Storage.
Intentando cargar el archivo: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet
Archivo 'yellow_tripdata_2021-02.parquet' subido exitosamente a Azure Blob Storage.
Intentando cargar el archivo: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet
Archivo 'yellow_tripdata_2021-03.parquet' subido exitosamente a Azure Blob Storage.
Intentando cargar el archivo: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-04.parquet
Archivo 'yellow_tripdata_2021-04.parquet' subido exitosamente a Azure Blob Storage.
Intentando cargar el archivo: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-05.parquet
Archivo 'yellow_tripdata_2021-05.parquet' subido exitosamente a Azure Blob Storage.
Intentando cargar el archivo: 